In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [2]:
! pip install pyspark

In [0]:
from pyspark.sql import SparkSession

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import Row

# Initial SparkSession

In [0]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Assignment 6") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
import os
os.getcwd()

'/content'

# Reading dataset

In [7]:
#laod data as RDD
sc=spark.sparkContext
rdd_d=sc.textFile('/content/sample_data/california_housing_test.csv')
rdd_d.take(10)

[u'"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"',
 u'-122.050000,37.370000,27.000000,3885.000000,661.000000,1537.000000,606.000000,6.608500,344700.000000',
 u'-118.300000,34.260000,43.000000,1510.000000,310.000000,809.000000,277.000000,3.599000,176500.000000',
 u'-117.810000,33.780000,27.000000,3589.000000,507.000000,1484.000000,495.000000,5.793400,270500.000000',
 u'-118.360000,33.820000,28.000000,67.000000,15.000000,49.000000,11.000000,6.135900,330000.000000',
 u'-119.670000,36.330000,19.000000,1241.000000,244.000000,850.000000,237.000000,2.937500,81700.000000',
 u'-119.560000,36.510000,37.000000,1018.000000,213.000000,663.000000,204.000000,1.663500,67000.000000',
 u'-121.430000,38.630000,43.000000,1009.000000,225.000000,604.000000,218.000000,1.664100,67000.000000',
 u'-120.650000,35.480000,19.000000,2310.000000,471.000000,1341.000000,441.000000,3.225000,166900.000000',
 u'-122.840000,38.4000

In [8]:
#load the data as dataframe
data=spark.read.csv('/content/sample_data/california_housing_test.csv',header=True)
data.show(10)

+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population|households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|-122.050000|37.370000|         27.000000|3885.000000|    661.000000|1537.000000|606.000000|     6.608500|     344700.000000|
|-118.300000|34.260000|         43.000000|1510.000000|    310.000000| 809.000000|277.000000|     3.599000|     176500.000000|
|-117.810000|33.780000|         27.000000|3589.000000|    507.000000|1484.000000|495.000000|     5.793400|     270500.000000|
|-118.360000|33.820000|         28.000000|  67.000000|     15.000000|  49.000000| 11.000000|     6.135900|     330000.000000|
|-119.670000|36.330000|         19.000000|1241.000000|    244.000000| 850.000000|237.000000|     2.937500|      81700.

In [9]:
#remove the first row
f=rdd_d.first()
rdd_d=rdd_d.filter(lambda r:r!=f)
## convert rdd to dataframe
a=rdd_d.map(lambda x:x.split(","))
b=a.map(lambda p: Row(longitude=p[0],latitude=p[1],housing_median_age=p[2],total_rooms=p[3],total_bedrooms=p[4],population=p[5],households=p[6],median_income=p[7],median_house_value=p[8]))
rdd_df=spark.createDataFrame(b)
rdd_df.show(10)


+----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
|households|housing_median_age| latitude|  longitude|median_house_value|median_income| population|total_bedrooms|total_rooms|
+----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
|606.000000|         27.000000|37.370000|-122.050000|     344700.000000|     6.608500|1537.000000|    661.000000|3885.000000|
|277.000000|         43.000000|34.260000|-118.300000|     176500.000000|     3.599000| 809.000000|    310.000000|1510.000000|
|495.000000|         27.000000|33.780000|-117.810000|     270500.000000|     5.793400|1484.000000|    507.000000|3589.000000|
| 11.000000|         28.000000|33.820000|-118.360000|     330000.000000|     6.135900|  49.000000|     15.000000|  67.000000|
|237.000000|         19.000000|36.330000|-119.670000|      81700.000000|     2.937500| 850.000000|    244.000000|1241.

In [10]:
# convert dataframe to rdd
rdd1=rdd_df.rdd
rdd1

MapPartitionsRDD[29] at javaToPython at NativeMethodAccessorImpl.java:0

In [11]:
# convert spark dataframe to pandas dataframe
pandas=data.toPandas()
pandas

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.050000,37.370000,27.000000,3885.000000,661.000000,1537.000000,606.000000,6.608500,344700.000000
1,-118.300000,34.260000,43.000000,1510.000000,310.000000,809.000000,277.000000,3.599000,176500.000000
2,-117.810000,33.780000,27.000000,3589.000000,507.000000,1484.000000,495.000000,5.793400,270500.000000
3,-118.360000,33.820000,28.000000,67.000000,15.000000,49.000000,11.000000,6.135900,330000.000000
4,-119.670000,36.330000,19.000000,1241.000000,244.000000,850.000000,237.000000,2.937500,81700.000000
5,-119.560000,36.510000,37.000000,1018.000000,213.000000,663.000000,204.000000,1.663500,67000.000000
6,-121.430000,38.630000,43.000000,1009.000000,225.000000,604.000000,218.000000,1.664100,67000.000000
7,-120.650000,35.480000,19.000000,2310.000000,471.000000,1341.000000,441.000000,3.225000,166900.000000
8,-122.840000,38.400000,15.000000,3080.000000,617.000000,1446.000000,599.000000,3.669600,194400.000000
9,-118.020000,34.080000,31.000000,2402.000000,632.000000,2830.000000,603.000000,2.333300,164200.000000


# Task

In [12]:
#Task 1 ---Select first 10 rows of dataset
rdd_df.show(10)

+----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
|households|housing_median_age| latitude|  longitude|median_house_value|median_income| population|total_bedrooms|total_rooms|
+----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
|606.000000|         27.000000|37.370000|-122.050000|     344700.000000|     6.608500|1537.000000|    661.000000|3885.000000|
|277.000000|         43.000000|34.260000|-118.300000|     176500.000000|     3.599000| 809.000000|    310.000000|1510.000000|
|495.000000|         27.000000|33.780000|-117.810000|     270500.000000|     5.793400|1484.000000|    507.000000|3589.000000|
| 11.000000|         28.000000|33.820000|-118.360000|     330000.000000|     6.135900|  49.000000|     15.000000|  67.000000|
|237.000000|         19.000000|36.330000|-119.670000|      81700.000000|     2.937500| 850.000000|    244.000000|1241.

In [13]:
#Task 2---show the schema of the dataset
rdd_df.printSchema()

root
 |-- households: string (nullable = true)
 |-- housing_median_age: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- median_house_value: string (nullable = true)
 |-- median_income: string (nullable = true)
 |-- population: string (nullable = true)
 |-- total_bedrooms: string (nullable = true)
 |-- total_rooms: string (nullable = true)



In [14]:
# Task 3---Group by and get max, min, count of a column in the dataset
rdd_df.groupBy('households').count().show()
rdd_df.select(max('population'),min('population')).show()

+-----------+-----+
| households|count|
+-----------+-----+
| 873.000000|    1|
| 474.000000|    4|
| 442.000000|    4|
| 966.000000|    1|
| 165.000000|    3|
| 148.000000|    2|
| 836.000000|    2|
|1064.000000|    1|
| 979.000000|    3|
|1019.000000|    1|
| 296.000000|    5|
| 775.000000|    3|
| 665.000000|    3|
| 603.000000|    3|
| 539.000000|    3|
| 611.000000|    2|
| 977.000000|    1|
| 461.000000|    2|
| 508.000000|    2|
| 986.000000|    1|
+-----------+-----+
only showing top 20 rows

+---------------+---------------+
|max(population)|min(population)|
+---------------+---------------+
|     999.000000|     100.000000|
+---------------+---------------+



In [15]:
#Task 4---Filter your dataset by some conditions based on your column
rdd_df.filter(rdd_df['population']>500).show()

+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
| households|housing_median_age| latitude|  longitude|median_house_value|median_income| population|total_bedrooms|total_rooms|
+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
| 606.000000|         27.000000|37.370000|-122.050000|     344700.000000|     6.608500|1537.000000|    661.000000|3885.000000|
| 277.000000|         43.000000|34.260000|-118.300000|     176500.000000|     3.599000| 809.000000|    310.000000|1510.000000|
| 495.000000|         27.000000|33.780000|-117.810000|     270500.000000|     5.793400|1484.000000|    507.000000|3589.000000|
| 237.000000|         19.000000|36.330000|-119.670000|      81700.000000|     2.937500| 850.000000|    244.000000|1241.000000|
| 204.000000|         37.000000|36.510000|-119.560000|      67000.000000|     1.663500| 663.000000|    213.0000

In [16]:
#Task 5---Apply group by with having clause
rdd_df.groupBy('median_income').agg(sum('households')).alias('sum_households').show()

+-------------+---------------+
|median_income|sum(households)|
+-------------+---------------+
|     4.638900|          169.0|
|     4.693000|          985.0|
|     5.898000|          285.0|
|    10.598100|          484.0|
|     0.740300|          350.0|
|     3.272700|          456.0|
|     4.931800|          323.0|
|     2.306800|          273.0|
|     2.401000|          337.0|
|     4.163000|          284.0|
|     3.209100|          603.0|
|     1.442700|          390.0|
|     2.021700|          472.0|
|     4.736100|         1224.0|
|     4.786100|          421.0|
|     2.853300|           71.0|
|     5.273000|          273.0|
|     1.000000|           51.0|
|     4.033300|          622.0|
|     3.956600|          454.0|
+-------------+---------------+
only showing top 20 rows



In [17]:
#Task 6 ---Apply order by
rdd_df.select('*').orderBy('households').show()

+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
| households|housing_median_age| latitude|  longitude|median_house_value|median_income| population|total_bedrooms|total_rooms|
+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
|  10.000000|         26.000000|36.620000|-121.840000|     150000.000000|     2.225000|  27.000000|      8.000000|  32.000000|
| 100.000000|         29.000000|39.340000|-120.350000|      95800.000000|     4.027800| 337.000000|    474.000000|1986.000000|
| 100.000000|         35.000000|37.770000|-120.850000|      75000.000000|     2.458300| 261.000000|     96.000000| 404.000000|
|1000.000000|         29.000000|39.750000|-121.840000|      74500.000000|     1.728400|2053.000000|   1053.000000|4362.000000|
|1001.000000|          5.000000|33.760000|-117.540000|     160800.000000|     4.796500|3258.000000|   1035.0000

In [18]:
#Task 7---Select distinct records by a column
rdd_df.select('housing_median_age').distinct().show()

+------------------+
|housing_median_age|
+------------------+
|          1.000000|
|         37.000000|
|         32.000000|
|         47.000000|
|         45.000000|
|         11.000000|
|         18.000000|
|          3.000000|
|         17.000000|
|         33.000000|
|          4.000000|
|         19.000000|
|         44.000000|
|         39.000000|
|         23.000000|
|         34.000000|
|         25.000000|
|         15.000000|
|         51.000000|
|         22.000000|
+------------------+
only showing top 20 rows



In [30]:
#Task 8---Transform the data type of columns from int to string
from pyspark.sql.types import *
new_rdd_df=rdd_df.withColumn('households',rdd_df['households'].cast(StringType()))
new_rdd_df.printSchema()

root
 |-- households: string (nullable = true)
 |-- housing_median_age: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- median_house_value: string (nullable = true)
 |-- median_income: string (nullable = true)
 |-- population: string (nullable = true)
 |-- total_bedrooms: string (nullable = true)
 |-- total_rooms: string (nullable = true)

